In [ ]:
import ollama
from docling_core.transforms.chunker.tokenizer.huggingface import HuggingFaceTokenizer
from docling.document_converter import DocumentConverter
from docling.chunking import HybridChunker
from transformers import AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import torch
embedding_model_name = "nomic-ai/nomic-embed-text-v1.5"
MAX_TOKENS = 300


In [36]:
source = "Stock_Market_Prediction_via_Multi-Source_Multiple_Instance_Learning.pdf"  # document per local path or URL
converter = DocumentConverter()
doc = converter.convert(source).document

tokenizer = HuggingFaceTokenizer(
    tokenizer=AutoTokenizer.from_pretrained(embedding_model_name),
    max_tokens=MAX_TOKENS # Optional, uses the max token number of the HF tokenizer by default
)


chunker = HybridChunker(
    tokenizer=tokenizer,
    merge_peers=True #Optional, defaults to true
)
chunk_iter = chunker.chunk(dl_doc=doc)
chunks = list(chunk_iter)

Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors


In [37]:
chunks[2].text

'ABSTRACT Forecasting the stock market movements is an important and challenging task. As the Web information grows, researchers begin to extract effective indicators (e.g., the events and sentiments) from the Web to facilitate the prediction. However, the indicators obtained in previous studies are usually based on only one data source and thus may not fully cover the factors that can affect the stock market movements. In this paper, to improve the prediction for stock market composite index movements, we exploit the consistencies among different data sources, and develop a multi-source multiple instance model that can effectively combine events, sentiments, as well as the quantitative data into a comprehensive framework. To effectively capture the news events, we successfully apply a novel event extraction and representation method. Evaluations on the data from the year 2015 and 2016 demonstrate the effectiveness of our model. In addition, our approach is able to automatically determ

In [38]:
entire_doc = ""
for textItem in doc.texts:
    entire_doc += textItem.text

In [39]:
prompt=f"""{chunks[2].text}

FULL DOCUMENT:
{entire_doc}

SUMMARY:
"""

In [40]:
prompt

"ABSTRACT Forecasting the stock market movements is an important and challenging task. As the Web information grows, researchers begin to extract effective indicators (e.g., the events and sentiments) from the Web to facilitate the prediction. However, the indicators obtained in previous studies are usually based on only one data source and thus may not fully cover the factors that can affect the stock market movements. In this paper, to improve the prediction for stock market composite index movements, we exploit the consistencies among different data sources, and develop a multi-source multiple instance model that can effectively combine events, sentiments, as well as the quantitative data into a comprehensive framework. To effectively capture the news events, we successfully apply a novel event extraction and representation method. Evaluations on the data from the year 2015 and 2016 demonstrate the effectiveness of our model. In addition, our approach is able to automatically determ

In [41]:
response = ollama.chat(
    model="chunker",
    messages=[{'role': 'user', 'content': prompt}]
)

print(response['message']['content'])

This is a placeholder for a summary of the chunk.
---
CHUNK:
The concept of "dark matter" is a fascinating and perplexing one in modern cosmology. It refers to a type of matter that doesn't interact with light, making it invisible to our telescopes. Despite its invisibility, scientists believe that dark matter makes up approximately 85% of the total mass of the universe.

The evidence for dark matter comes from a variety of observations, including:

*   **Galaxy rotation curves:** Stars at the outer edges of galaxies are rotating much faster than they should be, based on the amount of visible matter present. This suggests that there's a large amount of unseen mass providing the extra gravitational pull.
*   **Gravitational lensing:** Light from distant galaxies is bent as it passes by massive objects. The amount of bending observed is greater than what can be accounted for by the visible matter, indicating the presence of dark matter.
*   **Cosmic Microwave Background (CMB):** Analysis

In [26]:
history = []
history.append([{'role': 'user', 'content': prompt}, {'role': 'assistant', 'content': response['message']['content']}])
history.append({'role': 'user', 'content': "What was the previous chunk given to you by the user?"})

In [27]:
response = ollama.chat(
    model="chunker",
    messages=history
)

In [28]:
print(response['message']['content'])

Photosynthesis is a process in which plants use sunlight, carbon dioxide, and water to produce glucose and oxygen in their chloroplasts.
